In [2]:
# Import necessary libraries
import os
import json
import fitz  # PyMuPDF
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import boto3
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define utility functions

def download_pdf(pdf_url, local_path="bill_text.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download PDF from {pdf_url}")

def get_bill_text(input_pdf_path, output_json_path, title):
    full_text = ""
    with fitz.open(input_pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            full_text += text + " "

    summary_data = {"title": title, "full_text": full_text}

    with open(output_json_path, 'w') as json_file:
        json.dump(summary_data, json_file, indent=4)

    return os.path.abspath(output_json_path)

def upload_to_s3(bucket_name, file_path):
    s3_client = boto3.client('s3')
    file_key = f"bill_details/{file_path.split('/')[-1]}"
    s3_client.upload_file(file_path, bucket_name, file_key, ExtraArgs={'ACL': 'public-read'})
    object_url = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
    return object_url

def fetch_bill_details(bill_page_url):
    base_url = 'https://www.flsenate.gov'
    response = requests.get(urljoin(base_url, bill_page_url))

    bill_details = {"title": "", "description": "", "pdf_path": "", "full_text": ""}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        bill_title_tag = soup.find('div', id='prevNextBillNav').find_next('h2')
        bill_description_tag = soup.find('p', class_='width80')
        bill_pdf_link = soup.find('a', class_='lnk_BillTextPDF')

        if bill_title_tag:
            bill_details["title"] = bill_title_tag.get_text(strip=True)
        if bill_description_tag:
            bill_details["description"] = bill_description_tag.get_text(strip=True)
        if bill_pdf_link:
            pdf_url = urljoin(base_url, bill_pdf_link['href'])
            local_pdf_path = download_pdf(pdf_url)
            bill_details["pdf_path"] = local_pdf_path
            json_path = get_bill_text(local_pdf_path, "output.json", bill_details["title"])
    s3_url = upload_to_s3('ddp-bills', json_path)
    bill_details["billTextPath"] = s3_url  # Store the Object URL
    return bill_details

# Define SQLAlchemy model for 'bill' table

Base = declarative_base()
class Bill(Base):
    __tablename__ = 'bill'
    id = Column(Integer, primary_key=True)
    govId = Column(String)
    billTextPath = Column(String)

# Database connection and insert functions

def connect_to_db(host, database, user, password, port=3306):
    db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(db_url, echo=True)
    Session = sessionmaker(bind=engine)
    return Session()

def insert_bill(session, govId, billTextPath):
    new_bill = Bill(govId=govId, billTextPath=billTextPath)
    session.add(new_bill)
    session.commit()
    return new_bill.id

# Main execution

db_host = 'ddp-api.czqcac8oivov.us-east-1.rds.amazonaws.com'
db_name = 'digital_democracy'
db_user = 'DataWithAlex'
db_password = '%Mineguy29'
db_port = 3306

session = connect_to_db(db_host, db_name, db_user, db_password, db_port)
if session:
    bill_page_url = "https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText"
    bill_details = fetch_bill_details(bill_page_url)
    bill_id = insert_bill(session, "test", bill_details["billTextPath"])
    session.close()
    print("Bill inserted successfully. Bill ID:", bill_id)
else:
    print("Database connection not established.")


2024-01-22 16:38:02,751 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-01-22 16:38:02,752 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:38:03,025 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-01-22 16:38:03,026 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:38:03,275 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-01-22 16:38:03,276 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:38:03,365 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:38:03,457 INFO sqlalchemy.engine.Engine INSERT INTO bill (`govId`, `billTextPath`) VALUES (%(govId)s, %(billTextPath)s)
2024-01-22 16:38:03,458 INFO sqlalchemy.engine.Engine [generated in 0.09174s] {'govId': 'test', 'billTextPath': 'https://ddp-bills.s3.amazonaws.com/bill_details/output.json'}
2024-01-22 16:38:03,645 INFO sqlalchemy.engine.Engine COMMIT
2024-01-22 16:38:03,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:38:03,998 INFO sqlalchemy.engine.Engine SEL

In [5]:
# Import necessary libraries
import os
import json
import fitz  # PyMuPDF
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import boto3
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import re  # Importing regular expression library

# Define utility functions

def download_pdf(pdf_url, local_path="bill_text.pdf"):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        return local_path
    else:
        raise Exception(f"Failed to download PDF from {pdf_url}")

def get_bill_text(input_pdf_path, output_json_path, title):
    full_text = ""
    with fitz.open(input_pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            full_text += text + " "

    summary_data = {"title": title, "full_text": full_text}

    with open(output_json_path, 'w') as json_file:
        json.dump(summary_data, json_file, indent=4)

    return os.path.abspath(output_json_path)

def upload_to_s3(bucket_name, file_path):
    s3_client = boto3.client('s3')
    file_key = f"bill_details/{file_path.split('/')[-1]}"
    s3_client.upload_file(file_path, bucket_name, file_key, ExtraArgs={'ACL': 'public-read'})
    object_url = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
    return object_url

def fetch_bill_details(bill_page_url):
    base_url = 'https://www.flsenate.gov'
    response = requests.get(urljoin(base_url, bill_page_url))

    bill_details = {"title": "", "description": "", "pdf_path": "", "full_text": ""}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        bill_title_tag = soup.find('div', id='prevNextBillNav').find_next('h2')
        bill_description_tag = soup.find('p', class_='width80')
        bill_pdf_link = soup.find('a', class_='lnk_BillTextPDF')

        if bill_title_tag:
            bill_details["title"] = bill_title_tag.get_text(strip=True)
        if bill_description_tag:
            bill_details["description"] = bill_description_tag.get_text(strip=True)
        if bill_pdf_link:
            pdf_url = urljoin(base_url, bill_pdf_link['href'])
            local_pdf_path = download_pdf(pdf_url)
            bill_details["pdf_path"] = local_pdf_path
            json_path = get_bill_text(local_pdf_path, "output.json", bill_details["title"])

    # Extracting govId from the title
    gov_id_match = re.search(r"HB \d+", bill_details["title"])
    if gov_id_match:
        bill_details["govId"] = gov_id_match.group(0)
    else:
        raise ValueError("GovId not found in bill title")

    s3_url = upload_to_s3('ddp-bills', json_path)
    bill_details["billTextPath"] = s3_url  # Store the Object URL
    return bill_details

# Define SQLAlchemy model for 'bill' table

Base = declarative_base()
class Bill(Base):
    __tablename__ = 'bill'
    id = Column(Integer, primary_key=True)
    govId = Column(String)
    billTextPath = Column(String)

# Database connection and insert functions

def connect_to_db(host, database, user, password, port=3306):
    db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(db_url, echo=True)
    Session = sessionmaker(bind=engine)
    return Session()

def insert_bill(session, govId, billTextPath):
    new_bill = Bill(govId=govId, billTextPath=billTextPath)
    session.add(new_bill)
    session.commit()
    return new_bill.id

# Main execution

db_host = 'ddp-api.czqcac8oivov.us-east-1.rds.amazonaws.com'
db_name = 'digital_democracy'
db_user = 'DataWithAlex'
db_password = '%Mineguy29'
db_port = 3306

session = connect_to_db(db_host, db_name, db_user, db_password, db_port)
if session:
    bill_page_url = "https://www.flsenate.gov/Session/Bill/2023/23/ByCategory/?Tab=BillText"
    bill_details = fetch_bill_details(bill_page_url)
    bill_id = insert_bill(session, bill_details["govId"], bill_details["billTextPath"])
    session.close()
    print("Bill inserted successfully. Bill ID:", bill_id)
else:
    print("Database connection not established.")


2024-01-22 16:55:44,703 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-01-22 16:55:44,704 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:55:45,100 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-01-22 16:55:45,101 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:55:45,240 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-01-22 16:55:45,241 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-22 16:55:45,484 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:55:45,580 INFO sqlalchemy.engine.Engine INSERT INTO bill (`govId`, `billTextPath`) VALUES (%(govId)s, %(billTextPath)s)
2024-01-22 16:55:45,582 INFO sqlalchemy.engine.Engine [generated in 0.09558s] {'govId': 'HB 23', 'billTextPath': 'https://ddp-bills.s3.amazonaws.com/bill_details/output.json'}
2024-01-22 16:55:45,767 INFO sqlalchemy.engine.Engine COMMIT
2024-01-22 16:55:46,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-22 16:55:46,102 INFO sqlalchemy.engine.Engine SE